<a href="https://colab.research.google.com/github/questmapping/Lombardia-Air-Quality/blob/main/Shareable_Qualita_Aria_Lombardia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
#https://github.com/xmunoz/sodapy
#https://www.dati.lombardia.it/Ambiente/Stazioni-qualit-dell-aria/ib47-atvt
#https://dev.socrata.com/foundry/www.dati.lombardia.it/ib47-atvt


try:
  from sodapy import Socrata
except:
  from pip._internal.main import main as pip
  pip(["install", "sodapy"])
  from sodapy import Socrata

Prima di proseguire con l'esecuzione del programma, devi reperire da qui https://www.dati.lombardia.it/profile/edit/developer_settings un APP Token. Registrati, entra e clicca su "crea nuova applicazione". Finita la creazione avrai a disposizione l'App Token da inserire qui sotto


In [ ]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
#client = Socrata("www.dati.lombardia.it", None)

# Example authenticated client (needed for non-public datasets):
# Get a Token: https://www.dati.lombardia.it/profile/edit/developer_settings
try:
    client = Socrata("www.dati.lombardia.it",
                      app_token="Get an APP Token and Put it Here")
    auth=True
except ValueError:
    auth=False
    api="Connection Error"

auth

True

Elenco dei Comuni che hanno i sensori installati dall'Arpa Lombardia

In [ ]:
results = client.get("ib47-atvt",select="comune")
comune_df = pd.DataFrame.from_records(results).drop_duplicates()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
  print(comune_df)

Qui di seguito inserire il Comune scelto dall'elenco

In [ ]:
print('Digita il nome del Comune Lombardo e premi Invio')
Comune = input('>')

Digita il nome del Comune Lombardo e premi Invio
>Mantova


Qui lanciamo un semplice test per vedere se ci sono sensori attivi

In [ ]:
results = client.get("ib47-atvt",select="idsensore",where="comune = '"+Comune+"'")
results_df = pd.DataFrame.from_records(results)
lista = list(results_df.idsensore)
print(f"Lista di {len(lista)} Sensori su {Comune}: {lista}\n")
#Query Sensori
sensori=""
for sensore in lista:
    sensori+="idsensore = '"+str(sensore)+"' or "

sensori = sensori[:-4]

#per lavorare le date dobbiamo usare le funzioni di SOQL SODA https://dev.socrata.com/docs/functions/#,
results = client.get("nicp-bhqi", select="idsensore,valore,stato,date_trunc_ymd(data) as day", where="("+sensori+") and stato = 'VA'", order="data DESC")
print(f"Controllo sensori attivi: {results}")

Qui Invece facciamo questa media di misurazione "empirica" per ottenere un semplice indicatore di qualità dell'aria

In [ ]:
# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
#Le Query seguono le definizioni di SoQL https://dev.socrata.com/docs/queries/
#per vedere i campi e i formati, guardiamo qui https://dev.socrata.com/foundry/www.dati.lombardia.it/ib47-atvt
#il primo parametro è l'identificativo dei dati json, gli altri parametri sono query
#sul sito lombardia troviamo i dati per i sensori aria: https://www.dati.lombardia.it/browse?category=Ambiente&tags=sensori_aria&provenance=official
if auth:
    results = client.get("ib47-atvt",select="idsensore",where="comune = '"+Comune+"'", limit=20)
    
    # Convert to pandas DataFrame
    results_df = pd.DataFrame.from_records(results)
    if not results_df.empty:
      lista = list(results_df.idsensore)
      #Query Sensori
      sensori=""
      for sensore in lista:
          sensori+="idsensore = '"+str(sensore)+"' or "
      sensori = sensori[:-4]
      #print(sensori)
      
      #quando abbiamo i sensori, cerchiamo i dati di oggi nel mese in corso https://www.dati.lombardia.it/Ambiente/Dati-sensori-aria/nicp-bhqi
      today = str(datetime.now().date() - timedelta(days=3)) #sottraiamo 3 gg per avere semplicemente la sicurezza che ci siano dei dati salvati
      #per lavorare le date dobbiamo usare le funzioni di SOQL SODA https://dev.socrata.com/docs/functions/#,
      results = client.get("nicp-bhqi", select="idsensore,valore,stato,date_trunc_ymd(data) as day", where="("+sensori+") and day = '"+today+"' and stato = 'VA'", order="data DESC")
      results_df = pd.DataFrame.from_records(results)
      if not results_df.empty:
        meanQuality=results_df["valore"].astype(float).mean()
        #print(meanQuality)
        print(today+ " " + Comune + ", media microgrammi per m3 rilevati: "+str(meanQuality))
        if meanQuality < 15:
          print("Qualità dell'aria buona")
        elif meanQuality < 30:
          print("Qualità dell'aria mediocre")
        else:
          print("Qualità dell'aria pessima")
      else:
        print(f"Dati validi per {Comune} non disponibili")
      
    else:
      print(f"Sensori per {Comune} non disponibili")
                

2021-04-22 Mantova, media microgrammi per m3 rilevati: 15.925
Qualità dell'aria mediocre
